In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

1. history에 대해 drop duplicate 후 drop column      
2. watch 에 대해서도 drop duplicate 후 drop column     
3. 이후 inner join       
4. 다른 dataset merge   

문제 : 3번부터 램이 터져나감    
대안 1. 나눠서 조인 (노가다...)   
대안 2. 더 나은 노트북 환경 찾기   

In [2]:
'''
의심의 여지 없이 중복 존재 
아래 코드 실행 시 groupby로 상당히 많이 묶이고, 최종 column 개수가 899021로 줄었음을 알 수 있음
''' 
# history.groupby(['profile_id','log_time','ss_id']).count().sort_values('album_id',ascending=False)[:60]

'\n의심의 여지 없이 중복 존재 \n아래 코드 실행 시 groupby로 상당히 많이 묶이고, 최종 column 개수가 899021로 줄었음을 알 수 있음\n'

# 시청 시작 데이터에서 row 중복 체크 - 1만 6천개 가량의 중복 확인. Drop

In [3]:
history = pd.read_csv('../input/lgground/history_data.csv')
history.drop_duplicates(subset=['profile_id','log_time','ss_id'])  #중복 row 제거

# row = 1005651 rows × 8 columns
# after drop duplicates 899021 rows × 8 columns
# 106630 개 row 중복

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer
0,3,20220301115653,20220301115719,MKID003,15,NaN,Y,N
1,3,20220301115653,20220301115809,MKID003,16,NaN,Y,N
2,3,20220301115653,20220301115958,MKID003,17,NaN,Y,N
3,3,20220301115653,20220301120118,MKID003,18,NaN,Y,N
4,3,20220301115653,20220301120229,MKID003,19,NaN,Y,N
...,...,...,...,...,...,...,...,...
1005646,33032,20220427155091,20220427155668,MKID003,381,NaN,Y,N
1005647,33032,20220427155091,20220427155680,MKID003,381,NaN,Y,N
1005648,33032,20220427155839,20220427155810,MKID003,125,NaN,Y,N
1005649,33032,20220427155706,20220427155838,MKID003,125,NaN,Y,N


In [4]:
# 'continuous_play','short_trailer','log_time' -- drop
# payment 결측 0으로 대체

# history.drop(labels=['log_time','act_target_dtl','continuous_play','short_trailer'],axis=1).fillna(0,inplace=True)

# LOG time 으로 duplicate 수정 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Watch에 대해서도 동일한 작업 수행 (duplicate drop, feature selection)

In [5]:
watch = pd.read_csv('../input/lgground/watch_e_data.csv')
# watch.drop_duplicates(subset=['profile_id','log_time','ss_id'])  # 중복 제거 
# watch.drop(labels=['log_time','act_target_dtl'],axis=1,inplace=True)  # column 드랍. 따로 처리할 결측치 없음

In [6]:
# full 병합 코드는 폭탄 그 자체ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
# 돌렸다 하면 터져용 

# datamerge = pd.merge(history,watch,how='inner',on='profile_id')
# datamerge.to_csv('../input/lgground')

*근데 여기서 문득 드는 생각*

1. 반드시 history와 watch를 묶어야 할까? 그러니까... serve할 유저 리스트만 history에서 가져오고, 제대로 된 neg/pos sampling이 가능한 watch 데이터에 대해서만 학습할 수도 있지 않나?     
애초에 history에서 유의미한 column은 **시청 시작 데이터에만 있는 profile_id & 그나마 payment 뿐이다**    
ㄴ *그나마 payment* : payment는 영상 시청의 장벽 같은 요소로 작동하기 때문에 (무료 영상이 유료 영상보다 많이 재생된다) sampling 혹은 추천 요소에 중요하게 고려할 만 하다.    
.       
           
이렇게 접근하면 어떤 방식이 가능해지냐면 < ss_id column도 날릴 수 있게 된다 :: 그래도 train-vaild를 위해서 날리진 말자 >    
1. 두 dataset에서 필요 없는 column을 drop하고       
2. history에만 있는 profile_id 가 있는 row를 추출해 watch에 concatenate한다.  [차집합만 추출](https://codealone.tistory.com/78)    
3. 추후 데이터셋 분할을 고려해, ss_id 기준으로 dataframe 정렬 sort.    
3. 위 데이터셋에 meta + profile 데이터셋 merge를 수행한다.    
     
> 최종적으로는 watch에 있는 중요한 값을 기본으로 가지고, serve대상인 유저목록까지 챙긴 dataset이 됨     
오케이 함 해보자구 

In [7]:
# history에만 있는 profile_id 추출하기 / Length: 8311
m_his = history['profile_id'].drop_duplicates() # 데이터셋 크기를 위해 profile_id column만 남김 
# watch에만 있는 profile_id 추출하기 / Length: 7658
m_wat = watch['profile_id'].drop_duplicates()
id_only_in_history = pd.merge(m_his,m_wat,how='outer',indicator=True
                ).query('_merge == "left_only"').drop(columns=['_merge'])
id_only_in_history_list = id_only_in_history['profile_id'].to_list()
id_only_in_history_rows = history[history['profile_id'].isin(id_only_in_history_list)]
id_only_in_history_rows = id_only_in_history_rows.drop_duplicates(subset=['profile_id','ss_id','album_id']) # 중복 row 드랍까지
id_only_in_history_rows 

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer
573,12,20220301185040,20220301185149,MKID003,352,NaN,N,N
575,12,20220318074859,20220318084940,MKID003,353,NaN,Y,N
576,12,20220318074859,20220318090447,MKID003,354,NaN,Y,N
577,12,20220318074859,20220318094103,MKID003,355,NaN,Y,N
578,12,20220427193517,20220427193544,MKID003,356,NaN,Y,N
...,...,...,...,...,...,...,...,...
1005289,32979,20220424101109,20220424101233,MKID003,17,NaN,Y,N
1005290,32979,20220424101109,20220424101309,MKID003,667,NaN,Y,N
1005291,32979,20220424101109,20220424101612,MKID003,5191,NaN,Y,N
1005480,33022,20220424105716,20220424110250,MKID003,1005,NaN,Y,N


In [8]:
# data preprocessing
id_only_in_history_list = id_only_in_history['profile_id'].to_list()
id_only_in_history_rows = history[history['profile_id'].isin(id_only_in_history_list)]
id_only_in_history_rows = id_only_in_history_rows.drop_duplicates(subset=['profile_id','ss_id','album_id']) # 중복 row 드랍까지
id_only_in_history_rows.drop(columns=['continuous_play','short_trailer'],inplace=True) # 총 12407개 / watch : 892794 +12407 = 905201
dataset = watch.append(id_only_in_history_rows,sort=False).sort_values('ss_id').reset_index(drop=True)

In [10]:
dataset['total_time'].isnull().sum() #제대로 합쳐졌다면 결측이 12407개 있을 것
dataset['profile_id'].nunique() #제대로 합쳐졌다면 8311명일 것 
#완-벽

8311

In [20]:
dataset = dataset.drop_duplicates(subset=['profile_id','log_time','album_id'])
len(dataset['profile_id'].unique())

8311

In [11]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905201 entries, 0 to 905200
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   profile_id       905201 non-null  int64  
 1   ss_id            905201 non-null  int64  
 2   log_time         905201 non-null  int64  
 3   act_target_dtl   905201 non-null  object 
 4   album_id         905201 non-null  int64  
 5   watch_time       892794 non-null  float64
 6   total_time       892794 non-null  float64
 7   continuous_play  892794 non-null  float64
 8   payment          424 non-null     float64
dtypes: float64(4), int64(4), object(1)
memory usage: 62.2+ MB


In [13]:
# # 메타데이터 병합 
metadata = pd.read_csv('../input/lgground/meta_data.csv')
meta_p = pd.read_csv('../input/lgground/meta_data_plus.csv')
meta = pd.merge(metadata,meta_p,how='inner',on='album_id') #### meta 정보 최종 집합체 

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## payments는 따로 history에서 row를 떼어와서 watch로 붙이던가 해야 함 (의미 있는 정보값) 
이제는 붙일 수 있음 (profile 정보가 watch에 모여 있으므로) 
> album_id기준으로 innerjoin붙여버리면 됨! 

In [9]:
# pay_his = history[['profile_id','album_id','ss_id']].drop_duplicates()
# dataset = pd.merge(dataset,pay_his,how='inner',on=['profile_id','album_id','ss_id'])

## 잊지 말아야 할 것
> history의 nunique한 아이템 수 : 대략 2만건   
mata의 nunique한 아이템 수 : 대략 4만건   
== 2달간의 소비데이터에 등장조차 하지 않은 아이템이 2만개나 있다

# next step


profile_data.csv 의 경우, 성별 column 수치화 작업 필요  (근데 EDA 가볍게 하고 나서 해도 늦지 않음)     
    
사실 feature encoding 작업이 모델학습 전, 전반적으로 필요